# Sample Jupyter ProbLog notebook
This is a simple [python wrapper kernel](https://jupyter-client.readthedocs.io/en/latest/wrapperkernels.html) for [ProbLog](https://dtai.cs.kuleuven.be/problog/).

## Supported modes
Currently, only the default problog mode (inference) is supported.

# Basics
When a cell is executed, the statements in it are added to the same problog program. You can organize your program into different cells and execute them as if they were all part of a single program (remember that a cell must be executed for it to be added to your program). Executing a cell a second time would typically add the statements to the program again. To avoid these, we introduce cell id's.

## Cell Ids
If the first non-empty line in a cell starts with `%#`, the string that follows is considered the id of the cell.
Modifying a cell with an id and re-evaluating will emulate an update of the cell - all the previously evaluated static-clauses are removed from the program, and the contents of the cell is added again. Note that **side-effects will NOT be undone**.

In the following cell, we declare an annotated disjunction. Note that it has a cell-id, so you can modify it and re-evaluate to modify the program.

In [1]:
%# adding_facts

0.2::foo(2); 0.3::foo(3); 0.5::foo(5):- bar(Z).

## Queries
You can use the standard way of querying problog using the `query/1` and `evidence/1` (and `evidence/2`) predicates. 
In notebooks,  **queries and evidence will NOT be added to the program**.  Other statements (here, `bar/1`) will be.

In [2]:
%# standard_queries
bar(a).
0.7::bar(b).

evidence(\+bar(b)).

query(foo(X)).

(Cell queries) 
 evidence: {bar(b): False} 
 0.500000 foo(5) 

 0.300000 foo(3) 

 0.200000 foo(2)

### Inline queries
We also allow queries as you would type in the interactive-prompt (the 'shell' mode of problog). Any line that starts with `%?` is treated as an inline query. 
* You can have multiple such queries in a cell.
* Results are formatted slightly differently - first substitution and then the probability.

(Since this cell does not add anything to the program, a cell-id makes little difference.)

In [3]:

% You can do multiple queries, with different evidence.
? foo(X)|bar(b).
? foo(X), foo(Y), X\=Y.


? foo(X)|bar(b) 
 evidence: {bar(b): True} 
 0.750000 X=5 
 0.510000 X=3 
 0.360000 X=2 
 ? foo(X), foo(Y), X\=Y 
 0.210000 X=3, Y=5 
 0.210000 X=5, Y=3 
 0.140000 X=2, Y=5 
 0.140000 X=5, Y=2 
 0.084000 X=2, Y=3 
 0.084000 X=3, Y=2

### Efficiency note
All queries in a single cell are evaluated together. Queries in different cells are evaluated separately.
They do share the same problog program, hence side-effects from queries may cross over.

To demonstrate, We will use `assertz/1` to add a fact to the database, and query it in the next cell. Also note that running this multiple times will cause `moo(3)` to be added to the program multiple times, and hence change the result.

In [4]:
%# side_effects
:- use_module(library(assert)).
? assertz((0.5::moo(3))).

? assertz(0.5::moo(3)) 
 1.000000

In [5]:
%# observe_side_effect
? moo(X).

? moo(X) 
 0.500000 X=3